In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools 
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import datajoint as dj
from collections import defaultdict
import pycircstat as pycs
from tqdm import tqdm
import scipy.stats as stats


In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20


pinky = dj.create_virtual_module('pinky', 'microns_pinky')
ndap100 = dj.create_virtual_module('pinky_nda', 'microns_pinky_nda')
radtune = dj.create_virtual_module('pinky_radtune', 'microns_pinky_radtune')
spattune = dj.create_virtual_module('pinky_spattune', 'microns_pinky_spattune')
fc = dj.create_virtual_module('pinky_fc', 'microns_pinky_fc')


Connecting celiib@10.28.0.34:3306


OperationalError: (2003, "Can't connect to MySQL server on '10.28.0.34' (timed out)")

In [ ]:
#demonstrates that all the correlation tables have been fully run
print((len(fc.ContactCorrelationShaft()),
       len(fc.ContactCorrelationHead()),
       len(fc.ContactCorrelationAxon()),
       len(fc.ContactCorrelationSoma()),
       len(fc.ContactCorrelationUnused())))

In [ ]:
# segments that have dendrite verteces > restriction threshold

dendrite_labels = ["Apical","Basal","Oblique","Dendrite"]

#dj.U("segment_id").aggr(pinky.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels]
#pinky.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500"
"""
Gets the number of dendritic vertex counts for each nueron in excitatory and orphan table
"""
excitatory = dj.U("segment_id").aggr(pinky.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500",
                       dendrite_vertex_count="sum(n_vertex_indices)")

orphan = dj.U("segment_id").aggr(pinky.CompartmentOrphan.ComponentOrphan() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500",
                       dendrite_vertex_count="sum(n_vertex_indices)")

#restricts to only those neurons that don't have 15000 vertex in the dendritic label group
restriction_threshold = 15000
dendrite_restriction = ((excitatory & "dendrite_vertex_count >" + str(restriction_threshold)).proj() + 
                            (orphan & "dendrite_vertex_count >" + str(restriction_threshold)).proj())

dendrite_restriction

In [ ]:
#pinky.CompartmentFinal.ComponentFinal() - dendrite_restriction
pinky.CompartmentOrphan.ComponentOrphan() - dendrite_restriction

In [ ]:
#pinky.SegmentExclude.describe()
pinky.ExclusionCriteria()

In [ ]:
# Exclude synapses, segments and restrict by functional confidence 

#gets rid of any synapse excludeds
contact = pinky.SkeletonContact & pinky.CurrentSegmentation
synapse = (pinky.Synapse - pinky.SynapseExclude) & pinky.CurrentSegmentation

#gets rid of any segment excludes
segment = (pinky.Segment - pinky.SegmentExclude) & pinky.CurrentSegmentation
soma = (pinky.AllenSoma - pinky.SegmentExclude) & pinky.CurrentSegmentation

In [ ]:
# segments that have dendrite verteces > restriction threshold

dendrite_labels = ["Apical","Basal","Oblique","Dendrite"]

#dj.U("segment_id").aggr(pinky.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels]
#pinky.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500"
"""
Gets the number of dendritic vertex counts for each nueron in excitatory and orphan table
"""
excitatory = dj.U("segment_id").aggr(pinky.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500",
                       dendrite_vertex_count="sum(n_vertex_indices)")

orphan = dj.U("segment_id").aggr(pinky.CompartmentOrphan.ComponentOrphan() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500",
                       dendrite_vertex_count="sum(n_vertex_indices)")

#restricts to only those neurons that don't have 15000 vertex in the dendritic label group
restriction_threshold = 15000
dendrite_restriction = ((excitatory & "dendrite_vertex_count >" + str(restriction_threshold)).proj() + 
                            (orphan & "dendrite_vertex_count >" + str(restriction_threshold)).proj())

dendrite_restriction

## Orientation / Direction / Von fit correlation

In [ ]:
"""
Reminder: 
1) ContactCorrelation table is for each segment pairs
- n seg shared, union, and correlations of the vectors of presyn pairs

"""

In [ ]:
# obtain a list of all keys in fc.ContactCorrelation and 
df = pd.DataFrame(fc.ContactCorrelation.fetch())
attrs = []
for col in df.columns:
    attrs.append(str(col))


In [ ]:
spat_threshold=1.35
ori_confidence=0.5
von_p_value=0.05

In [ ]:
'von_p_value <= ' + str(von_p_value) 

In [ ]:
tuned = 'confidence > ' + str(ori_confidence)
# gets the number of segments with certain confidence and von_p_value less than threshold
rad_units = radtune.BestVonFit.Unit & segment & tuned & 'von_p_value <= ' + str(von_p_value) 
rad_units

"""
gets a combination of all the segments that pass the orientation threshold
"""

rad_unit_pairs = (rad_units.proj(segment_id1 = 'segment_id') * 
                rad_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'


rad_unit_pairs

In [ ]:
"""
gets the pairwise of significantly tuned neurons and their 
orientation and directional differences from BestVonCorr
and appends the correlation data to it

-- then puts it in a dataframe
"""

radsyncont_df = pd.DataFrame(((radtune.BestVonCorr & rad_unit_pairs) * 
                              fc.ContactCorrelationShaft.proj(*attrs, cont_seg_shared = 'n_seg_shared', cont_seg_union = 'n_seg_union', segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())
radsyncont_df

In [ ]:
# set up bins for difference in prefered orientation 
rad2deg = 180/np.pi
ori_edges = np.linspace(0, np.pi /2 , 5) #gets the edges for the orientation edges
#generates labels for orientation
oe = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in ori_edges]]) 
ori_labels = list(zip(oe[:-1], oe[1:]))#
#generates the orientation centers
ori_centers = np.round((ori_edges[1:] + ori_edges[:-1])/2 * rad2deg, decimals=2) 
ori_labels

In [ ]:
# set up bins for difference in preferred direction (difference between direction can be at most pi)
dir_edges = np.linspace(0, np.pi, 5)
de = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in dir_edges]])
dir_labels = list(zip(de[:-1], de[1:]))
dir_centers = np.round((dir_edges[1:] + dir_edges[:-1])/2 * rad2deg, decimals=2) 
dir_labels

In [ ]:
# set up bins for von_corr (creates 6 bins)
vc_edges = np.linspace(min(radsyncont_df['von_corr']), max(radsyncont_df['von_corr']), 7)
ve = list(['{:.1f}'.format(ee) for ee in vc_edges])
vc_labels = list(zip(ve[:-1], ve[1:]))
#puts nans on the 
vc_centers = np.hstack((np.nan, np.round((vc_edges[1:] + vc_edges[:-1])/2, decimals=2), np.nan)) 
vc_centers


#### ***** for some reason wraps nans around the middles but didn't do that for the orientations ****** ####

In [ ]:
# bin diff_pref_ori, diff_pref_dir, von_corr and puts in dataframe
"""
stores the bin center for the bin that the difference delongs in
np.digitize returns the bin index + 1 of the bin that the value would belong in
"""

radsyncont_df['bin_diff_pref_ori'] = ori_centers[(np.digitize(np.abs(radsyncont_df['diff_pref_ori']), ori_edges)) - 1]
radsyncont_df['bin_diff_pref_dir'] = dir_centers[(np.digitize(np.abs(radsyncont_df['diff_pref_dir']), dir_edges)) - 1]
#radsyncont_df['bin_diff_sharp'] = sharp_centers[(np.digitize(np.abs(radsyncont_df['diff_sharp']), sharp_edges))]
radsyncont_df['bin_von_corr'] = vc_centers[(np.digitize(radsyncont_df['von_corr'], vc_edges))]

radsyncont_df
#-1.032120	-1.032120

In [ ]:
radsyncont_df[["diff_pref_ori","bin_diff_pref_ori"]]

In [ ]:
# compute n_seg_shared/n_seg_union for both synapse and contact vs functional differences
radsyncont_df['cont_shared_percent'] = radsyncont_df['cont_seg_shared'] / radsyncont_df['cont_seg_union']
radsyncont_df['syn_intersect'] = radsyncont_df['n_seg_a_converted'] + radsyncont_df['n_seg_b_converted'] - radsyncont_df['n_seg_shared_converted']
radsyncont_df['syn_shared_percent'] = radsyncont_df['syn_intersect'] / radsyncont_df['n_seg_shared_converted']

## Receptive field

In [ ]:
"""
For each neuron has the sta_snr that is used to restrict the receptive fields
"""
spat_units = spattune.BestSTA.Loc & 'sta_snr > '+ str(spat_threshold) & (segment & (spattune.BestSTA.Confidence() & tuned))
spat_units


In [ ]:
"""
Gets all the pairs that have a significant receptive field (without any repeats)
"""

spat_unit_pairs = (spat_units.proj(segment_id1 = 'segment_id') * 
                  spat_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'
spat_unit_pairs
# spatsyncont_df = pd.DataFrame(((spattune.BestSTACorr & sig_unit_pairs) * 
#                               fc.ContactCorrelationHead.proj(*attrs, cont_seg_shared = 'n_seg_shared', cont_seg_union = 'n_seg_union', segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())


In [ ]:
spattune.BestSTACorr() & "segment_id1>=segment_id2" #shows that segmnet1 is always less than segment2

In [ ]:
"""
Restricts the receptive field correlation  table to only the significant pairs
Attaches the correlation statistics to the end

-- stores it in a dataframe
"""
spatsyncon_df = pd.DataFrame(((spattune.BestSTACorr & spat_unit_pairs) * 
                              fc.ContactCorrelationHead.proj(*attrs, cont_seg_shared = 'n_seg_shared', cont_seg_union = 'n_seg_union', segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())
spatsyncon_df

In [ ]:
#function that returns the bin edges that will set up n_bins using equal depth (aka percentile)
def perc_bins(vals,n_bins): return [np.percentile(vals,p) for p in np.linspace(0,100,n_bins+1)]


In [ ]:
# set up percentile bins for union_corr_r2

r2_edges = perc_bins(spatsyncon_df['union_corr_r2'].values, 7)
re = list(['{:.1f}'.format(ee) for ee in r2_edges])
r2_labels = list(zip(re[:-1], re[1:]))
r2_centers = np.hstack((np.nan, np.round((np.array(r2_edges[1:]) + np.array(r2_edges[:-1]))/2, decimals=2), np.nan))
r2_labels

In [ ]:
r2_centers

In [ ]:
# set up percentile bins for center_dist

dist_edges = perc_bins(spatsyncon_df['center_dist'].values, 7)
de = list(['{:.1f}'.format(ee) for ee in r2_edges])
dist_labels = list(zip(de[:-1], de[1:]))
dist_centers = np.hstack((np.nan, np.round((np.array(dist_edges[1:]) + np.array(dist_edges[:-1]))/2, decimals=2), np.nan))
spatsyncon_df['bin_center_dist'] = dist_centers[(np.digitize(spatsyncon_df['center_dist'], dist_edges))]

In [ ]:
spatsyncon_df['bin_center_dist'] = dist_centers[(np.digitize(spatsyncon_df['center_dist'], dist_edges))]
spatsyncon_df['bin_union_corr_r2'] = r2_centers[(np.digitize(spatsyncon_df['union_corr_r2'], r2_edges))]
spatsyncon_df['cont_shared_percent'] = spatsyncon_df['cont_seg_shared'] / spatsyncon_df['cont_seg_union']
spatsyncon_df['syn_intersect'] = spatsyncon_df['n_seg_a_converted'] + spatsyncon_df['n_seg_b_converted'] - spatsyncon_df['n_seg_shared_converted']
spatsyncon_df['syn_shared_percent'] = spatsyncon_df['syn_intersect'] / spatsyncon_df['n_seg_shared_converted']

In [ ]:
"""
Conclusion: the binning of the correlations was done correctly
"""

spatsyncon_df[['bin_union_corr_r2','union_corr_r2']] 



## Plots

In [ ]:
fc.ContactCorrelationHead.describe()
print("hello")

In [ ]:
# Exclude synapses, segments and restrict by functional confidence 

#gets rid of any synapse excludeds
contact = pinky.SkeletonContact & pinky.CurrentSegmentation
synapse = (pinky.Synapse - pinky.SynapseExclude) & pinky.CurrentSegmentation

#gets rid of any segment excludes
segment = (pinky.Segment - pinky.SegmentExclude) & pinky.CurrentSegmentation
soma = (pinky.AllenSoma - pinky.SegmentExclude) & pinky.CurrentSegmentation

In [ ]:
spat_threshold=1.35
ori_confidence=0.5
von_p_value=0.05

In [ ]:
# set up bins for difference in prefered orientation 
rad2deg = 180/np.pi
ori_edges = np.linspace(0, np.pi /2 , 5) #gets the edges for the orientation edges
#generates labels for orientation
oe = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in ori_edges]]) 
ori_labels = list(zip(oe[:-1], oe[1:]))#
#generates the orientation centers
ori_centers = np.round((ori_edges[1:] + ori_edges[:-1])/2 * rad2deg, decimals=2) 
ori_labels


In [ ]:
#get all of the significant units for the orientation
tuned = 'confidence > ' + str(ori_confidence)
# gets the number of segments with certain confidence and von_p_value less than threshold
rad_units = radtune.BestVonFit.Unit & segment & tuned & 'von_p_value <= ' + str(von_p_value) 
rad_units

"""
gets a combination of all the segments that pass the orientation threshold
"""

rad_unit_pairs = (rad_units.proj(segment_id1 = 'segment_id') * 
                rad_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'


rad_unit_pairs

In [ ]:
#get all of the significant units for receptive field:
"""
For each neuron has the sta_snr that is used to restrict the receptive fields
"""
spat_units = spattune.BestSTA.Loc & 'sta_snr > '+ str(spat_threshold) & (segment & (spattune.BestSTA.Confidence() & tuned))
spat_units
"""
Gets all the pairs that have a significant receptive field (without any repeats)
"""

spat_unit_pairs = (spat_units.proj(segment_id1 = 'segment_id') * 
                  spat_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'
spat_unit_pairs
# spatsyncont_df = pd.DataFrame(((spattune.BestSTACorr & sig_unit_pairs) * 
#                               fc.ContactCorrelationHead.proj(*attrs, cont_seg_shared = 'n_seg_shared', cont_seg_union = 'n_seg_union', segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())


In [ ]:
rels = [fc.ContactCorrelationHead, fc.ContactCorrelation]
labels = ['Head', 'Total']
colors = ['g', 'k']
conn = ['cont_shared_percent', 'syn_shared_percent', 'density_pearson_converted']

fig, ax = plt.subplots(3, 1, figsize=(6, 12))

#loop through all of the relationships in the list
for i in range(len(rels)):
    #gets the correlation tables with the significant orientation pairs and appends the correlation attributes
    """
    cont_seg_shared = 'n_seg_shared' -->n_presyns contacting onto both segment_id and segment_b
    cont_seg_union = 'n_seg_union' --> n_presyns contacting either segment_id or segment_b
    """
    radsyncont_df = pd.DataFrame(((radtune.BestVonCorr & rad_unit_pairs) * 
                              rels[i].proj(*attrs, cont_seg_shared = 'n_seg_shared', cont_seg_union = 'n_seg_union', segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())
    
    # set up bins for von_corr (creates 6 bins)
    vc_edges = np.linspace(min(radsyncont_df['von_corr']), max(radsyncont_df['von_corr']), 7)
    ve = list(['{:.1f}'.format(ee) for ee in vc_edges])
    vc_labels = list(zip(ve[:-1], ve[1:]))
    #puts nans on the 
    vc_centers = np.hstack((np.nan, np.round((vc_edges[1:] + vc_edges[:-1])/2, decimals=2), np.nan)) 
    vc_centers


    #### ***** for some reason wraps nans around the middles but didn't do that for the orientations ****** ####
    
    
   #computes the binned orientation preference difference
    radsyncont_df['bin_diff_pref_ori'] = ori_centers[(np.digitize(np.abs(radsyncont_df['diff_pref_ori']), ori_edges)) - 1]
    #computes the percentage of total contact segments shared in comparison to total union of contact presyns
    radsyncont_df['cont_shared_percent'] = radsyncont_df['cont_seg_shared'] / radsyncont_df['cont_seg_union']
    #the intersection of the shared contact segments
    radsyncont_df['syn_intersect'] = radsyncont_df['n_seg_a_converted'] + radsyncont_df['n_seg_b_converted'] - radsyncont_df['n_seg_shared_converted']
    #the intersection of contact presyns as a percentag of the union of contacts
    radsyncont_df['syn_shared_percent'] = radsyncont_df['syn_intersect'] / radsyncont_df['n_seg_shared_converted']
    
    for k in range(len(conn)):
        x_coords = ori_centers
        #finds the average of each of the following stats for each bin ['cont_shared_percent', 'syn_shared_percent', 'density_pearson_converted']
        y_coords = radsyncont_df.groupby('bin_diff_pref_ori').mean()[conn[k]]

        ax[k].plot(x_coords, y_coords, label=labels[i], color=colors[i])
        errors = radsyncont_df.groupby('bin_diff_pref_ori').sem()[conn[k]]  # compute SE
        ax[k].errorbar(x_coords, y_coords, yerr=errors, ecolor=colors[i], fmt=' ', zorder=-1, label=None)
        ax[k].set_xticks(ori_centers)
        ax[k].set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
        ax[k].tick_params(labelsize=14)
        ax[k].set_xlabel(r'$\Delta \theta$', fontsize=16)
        ax[k].spines['top'].set_color('none')
        ax[k].spines['right'].set_color('none')
ax[0].set_ylabel(r'$\langle$Proportion Shared Contacts$\rangle$', fontsize=16)
ax[1].set_ylabel(r'$\langle$Proportion Shared Synapses$\rangle$', fontsize=16)
ax[2].set_ylabel(r'$\langle$Synapse Density Correlation$\rangle$', fontsize=16)
ax[0].legend(loc=9, frameon=False, fontsize=14)
fig.tight_layout()
fig.savefig('figures/cont_syn_density.png', dpi=300)

In [ ]:
fc.ContactCorrelationShaft.describe()
print("hello")

In [ ]:
# orientatin tuning 

rels = [fc.ContactCorrelationShaft, fc.ContactCorrelationHead, fc.ContactCorrelationSoma, fc.ContactCorrelationAxon]
labels = ['Shaft', 'Head', 'Soma', 'Axon']
colors = ['r', 'g', 'b', 'y']

fig, ax = plt.subplots(1, 1, figsize=(7, 5))
    
for i in range(len(rels)):
    #gets the orientation difference between pairs
    radsyncont_df = pd.DataFrame(((radtune.BestVonCorr & spat_unit_pairs) * 
                              rels[i].proj(*attrs, cont_seg_shared = 'n_seg_shared', cont_seg_union = 'n_seg_union', segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())
    
    #calculates the binned difference in preference orientation
    
    radsyncont_df['bin_diff_pref_ori'] = ori_centers[(np.digitize(np.abs(radsyncont_df['diff_pref_ori']), ori_edges)) - 1]
    
    x_coords = ori_centers
    #groups by the binned difference in preferred orientation 
    #density_pearson_converted=null : float                        # Pearson correlation for n_synapse/postsyn_length rate for axon group with at least 1 conversion
    y_coords = radsyncont_df.groupby('bin_diff_pref_ori').mean()['density_pearson']
    
    ax.plot(x_coords, y_coords, label=labels[i], color=colors[i])

    errors = radsyncont_df.groupby('bin_diff_pref_ori').sem()['density_pearson_converted']  # compute SE
    ax.errorbar(x_coords, y_coords, yerr=errors, ecolor=colors[i], fmt=' ', zorder=-1, label=None)
    
ax.legend(frameon=False, loc=0, fontsize=12)
ax.tick_params(labelsize=12)
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.set_xticks(ori_centers)
ax.set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
ax.set_xlabel(r'$\Delta \theta$', fontsize = 14)
ax.set_ylabel(r'$\langle$Synapse Density Correlation$\rangle$', fontsize = 14)
fig.savefig('figures/density_four_classes.png', dpi=300)

In [ ]:
from collections import Counter
import seaborn as sns
sns.set()
pearson_list = np.array(radsyncont_df["density_pearson_converted"].tolist())
correlation_numbers = pearson_list[np.where(~np.isnan(pearson_list))[0].astype("int")]
sns.distplot(correlation_numbers)

In [ ]:
"""
Conclusion: the synaptic densities are pearson correlations and a majority are negative

"""

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax.plot(x_coords, y_coords, label=labels[i], color=colors[i])
errors = spatsyncont_df.groupby('bin_center_dist').sem()['density_pearson_converted']  # compute SE
ax.errorbar(x_coords, y_coords, yerr=errors, ecolor=colors[i], fmt=' ', zorder=-1, label=None)
    

In [ ]:
dist_labels

In [ ]:
spat_units = spattune.BestSTA.Loc & 'sta_snr > 1.5' & (segment & (spattune.BestSTA.Confidence() & tuned))
spat_unit_pairs = (spat_units.proj(segment_id1 = 'segment_id') * 
                  spat_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'

In [ ]:
# RF distance

rels = [fc.ContactCorrelationShaft, fc.ContactCorrelationHead, fc.ContactCorrelationSoma, fc.ContactCorrelationAxon]
labels = ['Shaft', 'Head', 'Soma', 'Axon']
colors = ['r', 'g', 'b', 'y']

#rels = [ta3p100.ContactCorrelation]
#labels = ['Total']
#colors = ['k']
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    
for i in range(len(rels)):
    spatsyncont_df = pd.DataFrame(((spattune.BestSTACorr & spat_unit_pairs) * 
                              rels[i].proj(*attrs, cont_seg_shared = 'n_seg_shared', cont_seg_union = 'n_seg_union', segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())
    
    dist_edges = np.linspace(min(spatsyncont_df['center_dist']), max(spatsyncont_df['center_dist']), 8)
    de = list(['{:.1f}'.format(ee) for ee in dist_edges])
    dist_labels = list(zip(de[:-1], de[1:]))
    dist_centers = np.hstack((np.nan, np.round((np.array(dist_edges[1:]) + np.array(dist_edges[:-1]))/2, decimals=2), np.nan))
    spatsyncont_df['bin_center_dist'] = dist_centers[(np.digitize(spatsyncont_df['center_dist'], dist_edges))]
    
    x_coords = dist_centers[1:-1]
    y_coords = spatsyncont_df.groupby('bin_center_dist').mean()['density_pearson_converted']
    #print(x_coords, y_coords)
    ax.plot(x_coords, y_coords, label=labels[i], color=colors[i])
    #ax.scatter(x_coords, y_coords, label=labels[i], color=colors[i])

    errors = spatsyncont_df.groupby('bin_center_dist').sem()['density_pearson_converted']  # compute SE
    ax.errorbar(x_coords, y_coords, yerr=errors, ecolor=colors[i], fmt=' ', zorder=-1, label=None)
    
ax.legend(frameon=False, loc=0, fontsize=12)
ax.tick_params(labelsize=12)
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.set_xticks(dist_centers[1:-1])
ax.set_xticklabels(['[{},{}]'.format(*a) for a in dist_labels], rotation=15)
ax.set_xlabel('RF distance', fontsize = 15)
ax.set_ylabel(r'$\langle$Synapse Density Correlation$\rangle$', fontsize = 15)
fig.savefig('figures/RFdistance_density_four_classes.png', dpi=300)

In [ ]:
# receptive field

functional = ['bin_center_dist']    
connectomics = ['cont_shared_percent', 'syn_shared_percent', 'binary_conversion_pearson',
       'binary_conversion_cosine', 'binary_conv_jaccard_ones_ratio',
       'binary_conv_jaccard_matching_ratio', 'conversion_pearson',
       'conversion_cosine', 'density_pearson', 'density_cosine',
       'synapse_volume_mean_pearson', 'synapse_volume_mean_cosine',
       'synapse_vol_density_pearson', 'synapse_vol_density_cosine',
       'binary_conversion_pearson_converted',
       'binary_conversion_cosine_converted',
       'binary_conv_jaccard_ones_ratio_converted',
       'binary_conv_jaccard_matching_ratio_converted',
       'conversion_pearson_converted', 'conversion_cosine_converted',
       'density_pearson_converted', 'density_cosine_converted',
       'synapse_volume_mean_pearson_converted',
       'synapse_volume_mean_cosine_converted',
       'synapse_vol_density_pearson_converted',
       'synapse_vol_density_cosine_converted']

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(len(connectomics), len(functional), figsize=(5*len(functional), 3*len(connectomics)))

for i, pair in enumerate(itertools.product(functional, connectomics)):
    #sns.pointplot(pair[0], pair[1], data = spatsyncont_df, ax=ax[i], ci=None) 
    
    x_coords = dist_centers[1:-1]
    y_coords = spatsyncont_df.groupby(pair[0]).mean()[pair[1]] 
    ax[i].plot(x_coords, y_coords)
    
    errors = spatsyncont_df.groupby(pair[0]).sem()[pair[1]]  # compute SE
    ax[i].errorbar(x_coords, y_coords, yerr=errors, ecolor='k', fmt=' ', zorder=-1)
    
    ax[i].set_title(connectomics[i], fontsize=15)

sns.despine(trim=True)
fig.tight_layout()
fig.savefig('figures/Head_union_corr.png', dpi=100)

In [ ]:
# plot n_seg_shared/n_seg_union for both synapse and contact vs functional differences

functional = ['bin_diff_pref_ori', 'bin_diff_pref_dir', 'bin_von_corr']    
connectomics = ['cont_shared_percent', 'syn_shared_percent', 'binary_conversion_pearson',
       'binary_conversion_cosine', 'binary_conv_jaccard_ones_ratio',
       'binary_conv_jaccard_matching_ratio', 'conversion_pearson',
       'conversion_cosine', 'density_pearson', 'density_cosine',
       'synapse_volume_mean_pearson', 'synapse_volume_mean_cosine',
       'synapse_vol_density_pearson', 'synapse_vol_density_cosine',
       'binary_conversion_pearson_converted',
       'binary_conversion_cosine_converted',
       'binary_conv_jaccard_ones_ratio_converted',
       'binary_conv_jaccard_matching_ratio_converted',
       'conversion_pearson_converted', 'conversion_cosine_converted',
       'density_pearson_converted', 'density_cosine_converted',
       'synapse_volume_mean_pearson_converted',
       'synapse_volume_mean_cosine_converted',
       'synapse_vol_density_pearson_converted',
       'synapse_vol_density_cosine_converted']

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(len(connectomics), len(functional), figsize=(5*len(functional), 3*len(connectomics)))

for i, pair in enumerate(itertools.product(functional, connectomics)):
    sns.pointplot(pair[0], pair[1], data = radsyncont_df, ax=ax[i%len(connectomics), i//len(connectomics)], ci=None) 
    
    # manually plot error bars
    x_coords = []
    y_coords = []
    for point_pair in ax[i%len(connectomics), i//len(connectomics)].collections:
        for x, y in point_pair.get_offsets():
            x_coords.append(x)
            y_coords.append(y)
    errors = radsyncont_df.groupby(pair[0]).sem()[pair[1]]  # compute SE
    
    ax[i%len(connectomics), i//len(connectomics)].errorbar(x_coords, y_coords, yerr=errors, ecolor='k', fmt=' ', zorder=-1)

for i in range(len(connectomics)):
    ax[i, 1].set_title(connectomics[i], fontsize=15)

#sns.pointplot('bin_diff_pref_ori', 'cont_shared_percent', ci=None, data = radsyncont_df, ax=ax[0,0], linestyles='--', color='k')    
#sns.pointplot('bin_diff_pref_dir', 'cont_shared_percent', ci=None, data = radsyncont_df, ax=ax[0,1], linestyles='--', color='k')    
#sns.pointplot('bin_von_corr', 'cont_shared_percent', ci=None, data = radsyncont_df, ax=ax[0,2], linestyles='--', color='k')    
#sns.pointplot('bin_diff_pref_ori', 'syn_shared_percent', ci=None, data = radsyncont_df, ax=ax[1,0], color='k')    
#sns.pointplot('bin_diff_pref_dir', 'syn_shared_percent', ci=None, data = radsyncont_df, ax=ax[1,1], color='k')    
#sns.pointplot('bin_von_corr', 'syn_shared_percent', ci=None, data = radsyncont_df, ax=ax[1,2], color='k')  
'''
l = ['Contact', 'Synapse']
for i in range(2):
    ax[i, 0].set_title('{} percent shared seg vs diff in orientation'.format(l[i]))
    ax[i, 0].set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
    ax[i, 0].set_xlabel(r'$\Delta \theta$')
    ax[i, 0].set_ylabel('$<Shared/Union>$')

    ax[i, 1].set_title('{} percent shared seg vs diff in direction'.format(l[i]))
    ax[i, 1].set_xticklabels(['{}°-{}°'.format(*a) for a in dir_labels])
    ax[i, 1].set_xlabel(r'$\Delta \theta$')
    ax[i, 1].set_ylabel('$<Shared/Union>$')

    ax[i, 2].set_title('{} percent shared seg vs von corr'.format(l[i]))
    ax[i, 2].set_xticklabels(['[{},{}]'.format(*a) for a in vc_labels])
    ax[i, 2].set_xlabel('Von corr')
    ax[i, 2].set_ylabel('$<Shared/Union>$')

'''
sns.despine(trim=True)
fig.tight_layout()
fig.savefig('figures/Head_confidence1.0_percent_shared_seg_by_functional_difference.png', dpi=100)